## Lab 9 Self practice

Objectives:
- understand the impact of number of layers in CNN, padding, strides, pooling (max vs average, etc.)

Let's laod the required libraries and packages

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

We will be working on the MNSIT dataset for our experiments.

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Let's write a function that will create model based on the arguments that are passed into the function. 
You have to write the the lines whose descriptions are given in the comments.

In [3]:
def create_model(
    number_of_layers=2, 
    number_of_conv_kernels=[32,16],
    kernel_size_conv = [(3,3), (3,3)],
    padding_conv="valid",
    strides_conv=(1,1),
    pool_size=(2,2),
    is_pooling_average=False):
    
    if len(number_of_conv_kernels) != number_of_layers:
        raise ValueError('Number of elements in the number_of_conv_kernels should be equal to number_of_layers')
    if len(kernel_size_conv) != number_of_layers:
        raise ValueError('Number of elements in the number_of_conv_kernels should be equal to number_of_layers')

    model = keras.Sequential()
    model.add(keras.Input(shape=input_shape))

    for i in range(number_of_layers):

        # todo: write one line that:
        #   - creates a Conv2D layer; set the number of kernels;
        #   - sets the kernel size, strides, padding based on the values from the arguments of the function and activation function as 'relu';
        #   - adds the created layer into the model
        model.add(layers.Conv2D(number_of_conv_kernels[i], kernel_size_conv[i], strides=strides_conv, padding=padding_conv, activation='relu'))


        # todo: write 4 line that will create either AveragePooling2D or MaxPooling2D
        #   based on the is_pooling_average value passed into the function
        #   and sets the pool size based on the argument passed into the function
        if is_pooling_average:
            model.add(layers.AveragePooling2D(pool_size=pool_size))
        else:
            model.add(layers.MaxPooling2D(pool_size=pool_size))

    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation="softmax"))
    return model

We will write a function that will compile and train the passed model and return the accuracy

In [4]:
def train_model_and_get_accuracy(model):
    batch_size = 128
    epochs = 2

    # todo: write one line of code to compile the model with:
    #   - the categorical_crossentropy loss function
    #   - use adam as optimizer
    #   - use 'accuracy' as metrics
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    # todo: write one line of code to train the model:
    #   - set the size of the batch size
    #   - also, set train / validation dataset split as 90 / 10
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]

We can create several different combinations of parameters to try CNN models in different situations. Try to change the parameters that will increase the model accuracy.

In [5]:
import json

param1 = dict(number_of_layers=2, 
    number_of_conv_kernels=[32,16],
    kernel_size_conv = [(3,3), (3,3)],
    padding_conv="valid",
    strides_conv=(1,1),
    pool_size=(2,2),
    is_pooling_average=False)

param2 = dict(number_of_layers=3, 
    number_of_conv_kernels=[32,16,16],
    kernel_size_conv = [(3,3), (3,3), (3,3)],
    padding_conv="valid",
    strides_conv=(1,1),
    pool_size=(2,2),
    is_pooling_average=False)

param3 = dict(number_of_layers=2, 
    number_of_conv_kernels=[32,16],
    kernel_size_conv = [(3,3), (3,3)],
    padding_conv="valid",
    strides_conv=(1,1),
    pool_size=(2,2),
    is_pooling_average=True)


param4 = dict(number_of_layers=2, 
    number_of_conv_kernels=[64,32],
    kernel_size_conv = [(3,3), (3,3)],
    padding_conv="same",
    strides_conv=(1,1),
    pool_size=(3,3),
    is_pooling_average=False)

param_list = [param1, param2, param3, param4]

model_list = [create_model(**param) for param in param_list]

result_list = [train_model_and_get_accuracy(model) for model in model_list]

final_result = []


for i in range(len(param_list)):
    item = {
        'model arguments': param_list[i],
        'trainable param count': model_list[i].count_params(),
        'accuracy': result_list[i]
    }
    final_result.append(item)

final_result_json = {
    'items': final_result
}

print(json.dumps(final_result_json, indent = 4))

Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7327 - loss: 0.9217 - val_accuracy: 0.9660 - val_loss: 0.1252
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9600 - loss: 0.1323 - val_accuracy: 0.9768 - val_loss: 0.0823
Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.6254 - loss: 1.1965 - val_accuracy: 0.9330 - val_loss: 0.2405
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9236 - loss: 0.2508 - val_accuracy: 0.9517 - val_loss: 0.1790
Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6926 - loss: 0.9847 - val_accuracy: 0.9542 - val_loss: 0.1655
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.9478 - loss: 0.1818 - val_accuracy: 0.9693 - val_loss: 0.1050
Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 16s 34ms/step - accuracy: 0.7184 - loss: 0.9331 - val_accuracy: 0.9675 - val_loss: 0.1042
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 17s 40ms/step - accuracy: 0.9674 - loss: 0.1076 - val_accuracy